In [38]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
%matplotlib qt5

In [39]:
file = open("data.bin", "r")
interleaved_data = np.fromfile(file, np.uint8)
file.close()
# the data is sampled at 2.4Mhz
fs = 1.3E6 #Carrier frequency of SDR
dt = 1/(fs) #Timestep between samples 
symbol_r = 100E3 #Symbol rate

In [40]:
# I used this trick to de-interleave the data. There are many other methods.
# The data is in the form " real imag real imag real imag..." (interleaved)
# We want two separate arrays - one containing the real data and one containing the imag. 
I_data_raw = interleaved_data[0:len(interleaved_data):2] # This keeps every second 
# sample, starting from index 0 (all of the even index values)
Q_data_raw = interleaved_data[1:len(interleaved_data):2] # This keeps every second
# sample, starting from index 1 (all of the odd index values)

# Note: There are other ways of doing the de-interleaving. One other way is by using a loop.

# As stated on Clickup, 127.5 is the zero value. We therefore need to subtract it
# to remove the offset and center the data around zero. We also have to divide both arrays by
# the largest value to normalize the data.

I_samples = (I_data_raw-127.5)/127.5
Q_samples = (Q_data_raw-127.5)/127.5

# Make the data complex.
complex_data = I_samples + 1j*Q_samples

# Plot the in-phase data.
plt.figure(1)
plt.plot(np.abs(complex_data))
plt.xlabel("Time Bins")
plt.ylabel("Normalized Amplitude")
plt.xlim(0,len(I_samples))
plt.title("In-Phase Data (5 Bursts: OOK, 4-ASK, DBPSK, DQPSK, D8PSK)")
plt.grid()
plt.show()

# OOK

In [41]:
OOK_I = I_samples[62200:85472]
OOK_Q = Q_samples[62200:85472]

#Making the data complex as demonstarted in esc_python_tut2.py
OOK_signal = OOK_I + 1j*OOK_Q
OOK_signal_abs = np.abs(OOK_signal)
#Normalizing the data
OOK_I = OOK_I / OOK_signal_abs.max()
OOK_Q = OOK_Q / OOK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
OOK_signal = OOK_I + 1j*OOK_Q

#Scatter Plot of the ASK data.
plt.figure(1)
plt.scatter(np.real(OOK_signal), np.imag(OOK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
freq = np.fft.fftfreq(len(OOK_signal),1/fs)

#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(OOK_I),1)
t=bins*dt


#FFT 
OOK_SIGNAL = np.fft.fft(OOK_signal)

OOK_MAG = 10*np.log10(np.abs(OOK_SIGNAL))

OOK_SIGNAL_ABS = np.fft.fft(np.abs(OOK_signal))

OOK_MAG_ABS = 10*np.log10(np.abs(OOK_SIGNAL_ABS))

#Symbol Syncronization

#PLL if there is time

#FFT Method
plt.figure(2)
plt.plot(freq,OOK_MAG_ABS)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |OOK_SIGNAL|")
plt.grid()
plt.show()



#Carrier frequency synchronization.


Φ_local = 0      #Local phase
f_local = 100E3  #Local frequency
delay = 10*dt

freq_sync = np.cos(((2*np.pi)*f_local*(t-delay))+Φ_local)

Y_t = freq_sync*OOK_signal

Y_t=sp.correlate(freq_sync,OOK_signal,mode='full')

print(Y_t.max())
Φ = np.angle(Y_t.max())

plt.figure(3)
plt.plot(freq,OOK_MAG_ABS,label='FFT|FFT(OOK_signal)|')
plt.legend(loc="upper right")
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.grid()
plt.show()




f_clk = 56400 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
f_clk = freq[freq_bin]
print('Clock Frequency:',f_clk,'Hz')


#Pahse dection


print('Φ:',Φ,'radians')
print('Frequency/Φ bin:',freq_bin)

#Clock generation
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(9)
plt.plot(t,clock_signal)
plt.plot(t,np.abs(OOK_signal))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time(s)")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()

#Extracting the down sampled data
data_bin = []
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)

for i in peak_bins[0]:
    symbol_data.append(OOK_signal[i])

plt.figure(5)
plt.plot(np.abs(symbol_data))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()        
   

    
#Normalizing the data
OOK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
OOK_Q_Symbol = np.imag(symbol_data) / np.abs(symbol_data).max()  
symbol_data = OOK_I_Symbol + 1j*OOK_Q_Symbol

#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.

#create a time array
dt = 1/(f_clk) #Timestep between samples 
bins=np.arange(0,len(OOK_I_Symbol),1)
t=bins*dt

#correction of the frequency offset
Δ_Φ = np.diff(np.angle(symbol_data))
M_Δ_Φ = np.median(Δ_Φ)
freq_off = M_Δ_Φ/((2*np.pi*1)/dt)
print('M_Δ_Φ:',M_Δ_Φ)
print("Frequency offset:",freq_off,'Hz')


symbol_data = symbol_data*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

freq = np.fft.fftfreq(len(OOK_I_Symbol),dt)

OOK_SIGNAL_SHIFT = np.fft.fft(symbol_data)
OOK_MAG_SHIFT = 10*np.log10(np.abs(OOK_SIGNAL_SHIFT))


plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK: Scatter plot, downsampled")
plt.grid()
plt.show()

correction = np.e**(-1j*np.angle(symbol_data[0]))
i=0
while i < len(symbol_data):
    symbol_data[i] = symbol_data[i]*complex(correction)
    i=i+1
#Normalizing the data
OOK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
OOK_Q_Symbol = np.imag(symbol_data) / np.abs(symbol_data).max()  
symbol_data = OOK_I_Symbol + 1j*OOK_Q_Symbol

plt.figure(8)
plt.scatter(np.real(symbol_data),np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1.5,1.5)
plt.ylabel("Quadrature")
plt.ylim(-1.5,1.5)
plt.title("ASK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()



plt.figure(10)
plt.plot(freq,OOK_MAG_SHIFT,label="Shifted")
plt.legend(loc="upper right")
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("Frequency offset correction")
plt.grid()
plt.show()


(2.288655396547477-0.17167033400301016j)
Clock Frequency: 56419.73186662083 Hz
Φ: -0.07486904996134303 radians
Frequency/Φ bin: 1010
M_Δ_Φ: 1.7433318036521548
Frequency offset: 4.917780797974713e-06 Hz


# 4-ASK

In [68]:
ASK_I = I_samples[90750:116200]
ASK_Q = Q_samples[90750:116200]

#Making the data complex as demonstarted in esc_python_tut2.py
ASK_signal = ASK_I + 1j*ASK_Q
ASK_signal_abs = np.abs(ASK_signal)
#Normalizing the data
ASK_I = ASK_I / ASK_signal_abs.max()
ASK_Q = ASK_Q / ASK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
ASK_signal = ASK_I + 1j*ASK_Q

#Scatter Plot of the ASK data.
plt.figure(1)
plt.scatter(np.real(ASK_signal), np.imag(ASK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
freq = np.fft.fftfreq(len(ASK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(ASK_I),1)
t=bins*dt


ASK_SIGNAL = np.fft.fft(ASK_signal)
ASK_SIGNAL_dB = 10*np.log10(np.abs(ASK_SIGNAL))

#Symbol Syncronization
ASK_SIGNAL_ABS = np.fft.fft(np.abs(ASK_SIGNAL))
ASK_SIGNAL_ABS_dB = 10*np.log10(np.abs(ASK_SIGNAL_ABS))

plt.figure(2)
plt.plot(bins,ASK_SIGNAL_ABS)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |ASK_SIGNAL|")
plt.grid()
plt.show()

#Thus we obtain the frequency of the clock by looking for the largest spike above the noise.

f_clk = 108640 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
f_clk = freq[freq_bin]
print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)

#Need to obtain the phase 
ASK_signal_phase = np.unwrap(np.angle(ASK_SIGNAL)) # "np.angle returns the equivalent of atan(imag/real)

plt.figure(3)
plt.plot(bins,ASK_signal_phase)
plt.xlabel("bins")
plt.ylabel("Phase (radians)")
plt.grid()
plt.show()

print('Phase:',ASK_signal_phase[freq_bin],'radians')

#Once the phase is obtained we can use this in a cosine function to create a clock that -
# we may sample the data from inorder to extract the indformation from the signal.

clock_signal = 0.8*np.cos((2*np.pi*t*f_clk)+ASK_signal_phase[freq_bin])
plt.figure(4)
plt.plot(t,clock_signal)
plt.plot(t,np.abs(ASK_signal))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()




#Extracting the down sampled data
data_bin = []
symbol_data = []

peak_bins = sp.find_peaks(clock_signal)


for i in peak_bins[0]:
    symbol_data.append(ASK_signal_shift[i])

plt.figure(5)
plt.plot(np.abs(symbol_data))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()        
   

    
#Normalizing the data
ASK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
ASK_Q_Symbol = np.imag(symbol_data) / np.abs(symbol_data).max()  
symbol_data = ASK_I_Symbol + 1j*ASK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK: Scatter plot, downsampled")
plt.grid()
plt.show()




#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.
ASK_SIGNAL = np.fft.fft(ASK_signal)
ASK_MAG = 10*np.log10(np.abs(ASK_SIGNAL))
plt.figure(7)
plt.plot(freq,ASK_MAG)

#correction of the frequency offset
phase=[]
for i in range(len(ASK_SIGNAL)):
    phase.append(int(np.angle(ASK_signal[i])))
phase_δ=[]
for j in range(len(phase)-1):
    temp=(phase[j+1]-phase[j])
    phase_δ.append(temp)
phase_median=np.median(phase_δ)

freq_off = phase_median/(2*np.pi*dt)
print("Frequency offset:",freq_off,'Hz')
ASK_signal_shift = ASK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

ASK_SIGNAL_SHIFT = np.fft.fft(ASK_signal_shift)
ASK_MAG_SHIFT = 10*np.log10(np.abs(ASK_SIGNAL_SHIFT))
plt.plot(freq,ASK_MAG_SHIFT)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of ASK_signal")
plt.grid()
plt.show()




#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol))

correction = np.e**(-1j*np.angle(symbol_data[0]))
i=0
while i < len(symbol_data):
    symbol_data[i] = symbol_data[i]*complex(correction)
    i=i+1
#Normalizing the data
ASK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
ASK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = ASK_I_Symbol + 1j*ASK_Q_Symbol

plt.figure(8)
plt.scatter(np.real(symbol_data),np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1.5,1.5)
plt.ylabel("Quadrature")
plt.ylim(-1.5,1.5)
plt.title("ASK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()


data=[]
symbol_data= symbol_data[10:len(symbol_data):1]
for i, j in enumerate(np.real(symbol_data)):
    if  -1 <j < - 0.5:
        data.append(0)
        data.append(0)
    if -0.5 < j < 0:
        data.append(0)
        data.append(1)
    if 0 < j < 0.5:
        data.append(1)
        data.append(1)
    if 0.5 < j < 1:
        data.append(1)
        data.append(0)     

    
    

ValueError: x and y must have same first dimension, but have shapes (25450,) and (12726,)